# Week 5 lecture: Concurrency!(2/6/25)
- complex and difficult to debug! Foundation of knowledge is very important.
- Use -> when you have a pointer to a structure.
- Use . when you have an instance of a structure.

### Concurrency(application concurency)
- has three main flavors/options:
    1- Processes
        - independent "mostly" isolated units of execution that run in their own memory space/execution enviornment (unlike threads) and can perform tasks concurrently
        - two(or more) different programs running at the same time on different cpu cores within your computer.
            - computers have virtualized cores so that things can run in parallel 
        - processes can talk to eachother through Inter-process communication (IPC) mechanisms like pipes, message queues, shared memory, or sockets.
    2- I/O(input/output(reading/writing)) Multiplexing (most complicated!! needed when you only have one core or process/thread)
        - within a single process thread you're writting code to handle mutliple I/O streams at one time (ex. web server application => handling many requests at once)
        - your code implements "select" logic to switch between I/O streams in a single proccess/thread 
    3- Threads (best of proccess and I/O approach)
        - A thread is the smallest unit of execution that lives within a process and a process can contain one of more threads. Threads within a process all share resources such as memory space, which allows for more efficient communication between threads
- Concurrency Vs Parrallelism
    - concurrency refers to the ability of a system to handle multiple tasks in an overlapping manner, not neccessarily simultaneously. It may involve switching between tasks quickly giving the illusion of parrallel execution.
    - Parallelism - occurs when tasks are executed at the same time, often on different cpu cores

- How a program is loaded and ran: executable file(ELF) -> load into memory (stack(automatic variable storage-> heap(dynamic variable storage)-> .data/.bss(read/write(storage for global and local scoped variables))-> .rodata(read-only(storage for constants))-> text(read-only(binary instructions for machine code)))) -> Linux task table(keeps track of tasks,process, thread) -> execution.

- A task in linux is a schedulable entity so a process is a task, a thread is a task.
    - getpid() -> get process id
    - Syscall(SYS_gettid) -> get thread id.
    - when a new process is started the pid==thread id so that task_id is equal to pid:pid. this is until more threads are created in the process and then the task_id is altered to represent the pid and the individual thread_id
- the OS scheduler manages how "work" ready to be executed gets scheduled on the CPU(cores). For every peice of work the scheduler loads it into memory, puts it in a queue, then goes into core selection deciding where it can be processed. A peice of work may not always be completed on first attempt it might be thrown back into the queue to be run again if something goes wrong.
- Linux uses the task_struct data structure to manage tasks 
#### Processes/threads
- creating a child process includes all read and write memory pages are set to a "soft" read-only to enable copy-on-write functionality. the child and parent process both point to the same place in physical memory which can be read by them with no consequense. 
    - steps: 
    1 - Copy on write memory locked
    2 - new task_struct/ memory allocation for child
    3 - If either process modifies the copy on write memory then (performance hit) the copy-on-write pages are copied so that each process has its own memory and its no longer shared -which is expensive.
- Fork/Exec pattern looks similar/ is a variation of, creating a child process
    - Using posix_spawn() replaces "fork/exec" into a one step process
        - posic_spawn immediately creates a new pid and loads the new executable into memory - basically copying parent process but the two are NOT linked 
    - execv(executable, args) will take my current process in my executable and replace it with this process(executable)
        - only returns a value from the process when an error occurs

    - exec without fork simply deletes the original process and allows the process in the exec call to take over.

- when a new child thread is created it gets its own execution context(a task_struct) however, all of the pages are shared between the parent and the thread except for the stack as the thread gets its own stack immediately after being created
    - they share the same virtual memory space including global variables and the same heap space BUT they have their own stacks that hold localized variables
- when using fork() in C your code splits and you must implement logic for these processes
    - you can check if a process is a child or the parent by:
        - f_result = fork() => if f_result is 0 its a child and if 1 it is the parent
    - fork() in the parent returns the pid of the child.
    - wait() suspends execution of the current thread till ONE of its children terminates (returns).
    - you can check your child's exit status using a macro "WEXITSTATUS(c_result)" where c_result represents your child process
#### Fork/exec
- exec allows us have another process take over the current one
- If a child process outlives the parent process it becomes a zombie(orphaned) and the parent process gets reassigned to 1.

#### Whats a Shell
- a shell is a type of user interface for interacting with an operating system - its purose is to broker inputs and outputs to built-in-shell commands or binaries.
- a shell are also a part of "headless"(no-human-interaction) processes like cron jobs ( in linux cron jobs run binaries on a schedule automatically even if you are not logged in) - these files still run throught the context of a shell

